In [14]:
import numpy as np
import torch
import torchvision
import matplotlib.pyplot as plt
from time import time
from torchvision import datasets, transforms
from torch import nn, optim
import torch.nn.functional as F
from threading import Thread

In [15]:
transform = transforms.Compose([transforms.ToTensor(),
                              transforms.Normalize((0.5,), (0.5,)),
                              ])
trainset = datasets.MNIST('./data', download=True, train=True, transform=transform)
valset = datasets.MNIST('./data', download=True, train=False, transform=transform)

class Dataset:
    def __init__(self, trainset):
        self.trainset = trainset

    def job(self, num_workers):
        print('start job with {} workers'.format(num_workers))
        dataLoader = torch.utils.data.DataLoader(dataset=self.trainset,
                                     batch_size=64,
                                     shuffle=True,
                                     num_workers=num_workers)
        for images, labels in enumerate(dataLoader):
            print(images, labels)
            print('{} | batch id = {}/{}'.format(num_workers, i+1, len(dataLoader)))

trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True, num_workers=2)
valloader = torch.utils.data.DataLoader(valset, batch_size=64, shuffle=True)

In [16]:
'''dataiter = iter(trainloader)
images, labels = dataiter.next()

print(images.shape)
print(labels.shape)'''
dataset = Dataset(trainset)

In [17]:
filter_size1 = 5  # Convolution filters are 5 x 5 pixels.
num_filters1 = 16  # There are 16 of these filters.
stride1 = 1  # The stride of the sliding window

# 2nd Convolutional Layer
filter_size2 = 5  # Convolution filters are 5 x 5 pixels.
num_filters2 = 32  # There are 32 of these filters.
stride2 = 1  # The stride of the sliding window

num_classes = 10

class Model(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, num_filters1, kernel_size=(filter_size1, filter_size1))
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(num_filters1, num_filters2, kernel_size=(filter_size2, filter_size2))
        self.fc1 = nn.Linear(512, 128)
        self.fc2 = nn.Linear(128, num_classes)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x


model = Model()

In [18]:
criterion = nn.CrossEntropyLoss()
images, labels = next(iter(trainloader))
#images = images.view(images.shape[0], -1)
print(images.shape)

logps = model(images) #log probabilities
loss = criterion(logps, labels) #calculate the NLL loss

torch.Size([64, 1, 28, 28])


In [19]:
optimizer = optim.Adam(model.parameters(), lr=0.001)
time0 = time()
epochs = 15
for e in range(epochs):
    running_loss = 0
    for images, labels in trainloader:

        optimizer.zero_grad()

        output = model(images)
        loss = criterion(output, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
    print("Epoch {} - Training loss: {}".format(e, running_loss/len(trainloader)))
    print("\nTraining Time (in seconds) =",(time()-time0))

Epoch 0 - Training loss: 0.16828670530224532

Training Time (in seconds) = 12.23630952835083
Epoch 1 - Training loss: 0.04959093625106211

Training Time (in seconds) = 23.755156993865967
Epoch 2 - Training loss: 0.03508927408579254

Training Time (in seconds) = 35.593239545822144


KeyboardInterrupt: 

Traceback (most recent call last):
  File "/usr/lib/python3.7/multiprocessing/queues.py", line 242, in _feed
    send_bytes(obj)
  File "/usr/lib/python3.7/multiprocessing/connection.py", line 200, in send_bytes
    self._send_bytes(m[offset:offset + size])
  File "/usr/lib/python3.7/multiprocessing/connection.py", line 404, in _send_bytes
    self._send(header + buf)
  File "/usr/lib/python3.7/multiprocessing/connection.py", line 368, in _send
    n = write(self._handle, buf)
BrokenPipeError: [Errno 32] Broken pipe


In [ ]:
images, labels = next(iter(valloader))

with torch.no_grad():
    logps = model(images)

ps = torch.exp(logps)
probab = list(ps.numpy()[0])
print("Predicted Digit =", probab.index(max(probab)))
view_classify(img.view(1, 28, 28), ps)

In [ ]:
correct = 0
total = 0
# since we're not training, we don't need to calculate the gradients for our outputs
with torch.no_grad():
    for data in valloader:
        images, labels = data
        # calculate outputs by running images through the network
        outputs = model(images)
        # the class with the highest energy is what we choose as prediction
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))